##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantization aware training in Keras example

## Overview

Welcome to an end-to-end example for *quantization aware training*.

### Other pages
For an introduction to what quantization aware training is and to determine if you should use it (including what's supported), see the [overview page](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).

To quickly find the APIs you need for your use case (beyond fully-quantizing a model with 8-bits), see the
[comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md).

### Summary

In this tutorial, you will:

1.   Train a `keras` model for MNIST from scratch.
2.   Fine tune the model by applying the quantization aware training API, see the accuracy, and
     export a quantization aware model.
3.   Use the model to create an actually quantized model for the TFLite
     backend.
4.   See the persistence of accuracy in
     TFLite and a 4x smaller model. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

## Setup

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf

from tensorflow_model_optimization.python.core.keras.compat import keras

## Train a model for MNIST without quantization aware training

In [ ]:
# Load CIFAR dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28,28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)

1688/1688 [==============================] - 14s 6ms/step - loss: 0.2742 - accuracy: 0.9230 - val_loss: 0.1157 - val_accuracy: 0.9705


## Clone and fine-tune pre-trained model with quantization aware training


### Define the model

You will apply quantization aware training to the whole model and see this in the model summary. All layers are now prefixed by "quant".

Note that the resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8). The sections after show how to create a quantized model from the quantization aware one.

In the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md), you can see how to quantize some layers for model accuracy improvements.

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_reshape (QuantizeWra  (None, 28, 28, 1)         1         
 pperV2)                                                         
                                                                 
 quant_conv2d (QuantizeWrap  (None, 26, 26, 12)        147       
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 13, 13, 12)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_flatten (QuantizeWra  (None, 2028)              1

### Train and evaluate the model against baseline

To demonstrate fine tuning after training the model for just an epoch, fine tune with quantization aware training on a subset of the training data.

In [ ]:
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

2/2 [==============================] - 2s 288ms/step - loss: 0.1329 - accuracy: 0.9644 - val_loss: 0.1475 - val_accuracy: 0.9700


For this example, there is minimal to no loss in test accuracy after quantization aware training, compared to the baseline.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9664999842643738
Quant test accuracy: 0.9684000015258789


In [ ]:
model.layers[1].get_weights()

[array([[[[ 0.19930887,  0.32900944, -0.10285777,  0.15539318,
            0.04698373,  0.21746336,  0.15988956,  0.11993229,
            0.09372851, -0.42971954, -0.2648204 ,  0.4658565 ]],
 
         [[ 0.0464725 ,  0.05443647,  0.13574058, -0.03798673,
            0.07129293, -0.26029715, -0.10431486,  0.03323695,
            0.5385716 , -0.3865914 ,  0.05107578,  0.27737743]],
 
         [[ 0.29978737, -0.22625557,  0.27800846,  0.2240177 ,
            0.11337798, -0.23008159, -0.33084694, -0.21541525,
            0.5794873 ,  0.52377313,  0.44964314,  0.50095606]]],
 
 
        [[[ 0.10419063,  0.26376984, -0.17582765,  0.01501086,
           -0.13573734,  0.533272  ,  0.15568323,  0.1725869 ,
            0.17633526,  0.29533795, -0.33666766,  0.02940423]],
 
         [[ 0.27235177,  0.18503554,  0.2905227 ,  0.3311699 ,
            0.26667956, -0.07102464,  0.22441672,  0.28607878,
            0.13126917, -0.9098858 , -0.7793021 ,  0.13778478]],
 
         [[ 0.12530982,  0.00371

In [ ]:
q_aware_model.layers[2].get_weights()[0]

array([[[[ 0.20051304,  0.33084416, -0.10207862,  0.15704349,
           0.04813359,  0.21926825,  0.16059709,  0.12163318,
           0.0917767 , -0.43155318, -0.2668027 ,  0.46406344]],

        [[ 0.04787507,  0.05604396,  0.1368406 , -0.03628237,
           0.0728443 , -0.25847775, -0.10385922,  0.03493252,
           0.53659517, -0.38820577,  0.0490934 ,  0.27551204]],

        [[ 0.3010529 , -0.22542192,  0.27913532,  0.225594  ,
           0.11490519, -0.2285387 , -0.33269793, -0.21374658,
           0.5774919 ,  0.52179235,  0.44776404,  0.49905422]]],


       [[[ 0.10472445,  0.2653272 , -0.17520387,  0.01644931,
          -0.1347862 ,  0.5350148 ,  0.15634233,  0.1742427 ,
           0.1743566 ,  0.29390112, -0.3382628 ,  0.02750634]],

        [[ 0.27349487,  0.18643534,  0.2915327 ,  0.33268097,
           0.26806706, -0.06943978,  0.22514172,  0.28773364,
           0.12928429, -0.9113892 , -0.7808455 ,  0.13593383]],

        [[ 0.12643501,  0.00443227,  0.19381797,  0.2

## Create quantized model for TFLite backend

After this, you have an actually quantized model with int8 weights and uint8 activations.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([ 1, 28, 28], dtype=int32), 'shape_signature': array([-1, 28, 28], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:0', 'index': 19, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


You evaluate the quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9684
Quant TF test accuracy: 0.9684000015258789


## See 4x smaller model from quantization

You create a float TFLite model and then see that the quantized TFLite model
is 4x smaller.

In [ ]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 0.08069610595703125
Quantized model in Mb: 0.02361297607421875


## Conclusion

In this tutorial, you saw how to create quantization aware models with the TensorFlow Model Optimization Toolkit API and then quantized models for the TFLite backend.

You saw a 4x model size compression benefit for a model for MNIST, with minimal accuracy
difference. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

We encourage you to try this new capability, which can be particularly important for deployment in resource-constrained environments.
